## Feature Engineering

En esta etapa del proyecto, se prepararan los conjuntos datos necesarios para las funciones.

#### Importación de librerias necesarias:

In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import FuncionExtra as f

#### Lectura de los datasets modificados durante la etapa de ETL:

In [2]:
steam_games = pd.read_csv('../steam_games_limpio.csv')

user_reviews = pd.read_csv('../user_reviews_limpio.csv')

user_items = pd.read_csv('../user_items_limpio.csv')

### Sentiment Analysis

Creación de una nueva categoria en el dataset **user_reviews**, utilizando el analisis de sentimientos a traves de procesamiento de lenguaje natural. Esto asignara etiquetas a las opiniones siguiendo el siguiente patron:
- 0 : Se aplicara a opiniones consideradas negativas.
- 1 : Se aplicara a opiniones consideradas neutras.
- 2 : Se aplicara a opiniones consideradas positivas.

En casos donde no sea posible determinar la clasificación debido a la falta de reseña, se utilizara la etiqueta **1** (neutra).

In [3]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   funny        8151 non-null   object 
 1   posted       49186 non-null  float64
 2   last_edited  6140 non-null   object 
 3   item_id      59305 non-null  int64  
 4   helpful      59305 non-null  object 
 5   recommend    59305 non-null  bool   
 6   review       59275 non-null  object 
 7   user_id      59305 non-null  object 
 8   user_url     59305 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 3.7+ MB


In [4]:
f.tipo_datos(user_reviews)

,nombre_columna,tipo_dato
0,funny,"[<class 'float'>, <class 'str'>]"
1,posted,[<class 'float'>]
2,last_edited,"[<class 'float'>, <class 'str'>]"
3,item_id,[<class 'int'>]
4,helpful,[<class 'str'>]
5,recommend,[<class 'bool'>]
6,review,"[<class 'str'>, <class 'float'>]"
7,user_id,[<class 'str'>]
8,user_url,[<class 'str'>]


Modificación del tipo de dato de la columna **reviews** a string:

In [5]:
user_reviews['review'] = user_reviews['review'].astype(str)

Verificación:

In [6]:
type(user_reviews['review'][0])

str

Creación de la función para poder analizar la columna **review**:

In [7]:
def analizar_sentimiento(opinion):

    if isinstance(opinion, str) and len(opinion) > 0:

        analysis = TextBlob(opinion)
        
        sentiment = analysis.sentiment.polarity

        if sentiment < -0.1:
            return 0  
        
        elif sentiment > 0.1:
            return 2 
        
        else:
            return 1 
        
    else:
        return 1  

In [9]:
user_reviews['sentiment_analysis'] = user_reviews['review'].apply(analizar_sentimiento)

In [10]:
user_reviews.to_csv('../sentiment_analysis.csv', index=False)

Controlamos que el analisis sea el correcto:

In [11]:
user_reviews['review'][3]

'I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it\'s title, this is easily one of my GOTYs. You don\'t get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can\'t 360 noscope your friends, but what you can do is show them up with your bad ♥♥♥ dance moves and put them to shame as you show them what true fashion and color combinations are.I know this game says for kids but, this is easily for any age range and any age will have a blast playing this.8/8'

In [12]:
user_reviews['sentiment_analysis'][3]

2

Eliminamos la columna **review**, ya que esta sera reemplazada por la columna **sentiment_analysis** para facilitar el trabajo de los modelos de Machine Learning y el analisis de datos:

In [13]:
user_reviews = user_reviews.drop(columns = 'review')

In [14]:
user_reviews

,funny,posted,last_edited,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,NaN,2011.0,NaN,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
1,NaN,2011.0,NaN,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,NaN,2011.0,NaN,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,NaN,2014.0,NaN,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,2
4,NaN,2013.0,NaN,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
...,...,...,...,...,...,...,...,...,...
59300,NaN,NaN,NaN,70,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2
59301,NaN,NaN,NaN,362890,No ratings yet,True,76561198312638244,http://steamcommunity.com/profiles/76561198312...,2
59302,1 person found this review funny,NaN,NaN,273110,1 of 2 people (50%) found this review helpful,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1
59303,NaN,NaN,NaN,730,No ratings yet,True,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,2


## Creación de funciones

Ahora, crearemos las funciones para los endpoints que se consumirán en la API

#### Función **Developer**

*def Developer(desarrollador: str):* Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno:
{"Año": 2023, "Cantidad de Items": 50, "Contenido Free": 27%}

Cambiamos el nombre de la columna 'id', por 'item_id', para luego poder realizar las uniones necesarias:

In [15]:
steam_games.rename(columns = {'id':'item_id'}, inplace = True)

Del DataFrame 'steam_games' se extraen las columnas necesarias:

In [16]:
df_developer = steam_games[['price', 'release_year', 'developer', 'item_id']]

Verificamos el tipo de dato de cada columna del DataFrame obtenido:

In [17]:
f.tipo_datos(df_developer)

,nombre_columna,tipo_dato
0,price,[<class 'float'>]
1,release_year,[<class 'float'>]
2,developer,"[<class 'str'>, <class 'float'>]"
3,item_id,[<class 'float'>]


Cambiamos el tipo de dato de la columna 'release_year', de float a int:

In [18]:
df_developer['release_year'] = df_developer['release_year'].fillna(0).astype(int)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3248\1181055551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developer['release_year'] = df_developer['release_year'].fillna(0).astype(int)


Verificamos:

In [19]:
f.tipo_datos(df_developer)

,nombre_columna,tipo_dato
0,price,[<class 'float'>]
1,release_year,[<class 'int'>]
2,developer,"[<class 'str'>, <class 'float'>]"
3,item_id,[<class 'float'>]


Cambiamos el tipo de dato de la columna 'item_id' de float a int:

In [20]:
df_developer['item_id'] = pd.to_numeric(df_developer['item_id'], errors='coerce').fillna(0).astype(int)


C:\Users\ferna\AppData\Local\Temp\ipykernel_3248\1763806934.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developer['item_id'] = pd.to_numeric(df_developer['item_id'], errors='coerce').fillna(0).astype(int)


Verificamos:

In [21]:
f.tipo_datos(df_developer)

,nombre_columna,tipo_dato
0,price,[<class 'float'>]
1,release_year,[<class 'int'>]
2,developer,"[<class 'str'>, <class 'float'>]"
3,item_id,[<class 'int'>]


Eliminamos duplicados:

In [22]:
df_developer.drop_duplicates(inplace= True)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3248\1156679960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developer.drop_duplicates(inplace= True)


In [23]:
def Developer(desarrollador):
    developer = df_developer[df_developer['developer'] == desarrollador]

    items_por_anio = developer.groupby('release_year')['item_id'].count()

    free_por_anio = developer[developer['price'] == 0.0].groupby('release_year')['item_id'].count()

    porcentaje_gratuito = ((free_por_anio / items_por_anio) * 100).fillna(0).astype(int)

    return {
        'cantidad_por _año': free_por_anio.to_dict(),
        'porcentaje_gratis_por_anio': porcentaje_gratuito.to_dict()
    }

In [24]:
Developer('Valve')

{'cantidad_por _año': {2005: 1, 2007: 1, 2010: 1, 2012: 1, 2014: 7, 2016: 1},
 'porcentaje_gratis_por_anio': {1998: 0,
  1999: 0,
  2000: 0,
  2001: 0,
  2003: 0,
  2004: 0,
  2005: 100,
  2006: 0,
  2007: 33,
  2008: 0,
  2009: 0,
  2010: 50,
  2011: 0,
  2012: 50,
  2014: 100,
  2016: 100,
  2017: 0}}

In [85]:
df_developer.to_parquet('Data/developer.parquet', index=False)

#### Función **UserData**

*def UserData(user_id: str):* Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

Para comenzar, hacemos un rapido repaso de las columnas de los DataFrame que vamos a necesitar y el tipo de datos de cada columna para poder llevar a cabo la union de ambos DataFrame de ser necesario.

In [25]:
f.tipo_datos(user_items)

,nombre_columna,tipo_dato
0,item_id,[<class 'int'>]
1,item_name,[<class 'str'>]
2,playtime_forever,[<class 'int'>]
3,playtime_2weeks,[<class 'int'>]
4,user_id,[<class 'str'>]
5,items_count,[<class 'int'>]
6,steam_id,[<class 'int'>]
7,user_url,[<class 'str'>]


In [26]:
f.tipo_datos(steam_games)

,nombre_columna,tipo_dato
0,publisher,"[<class 'str'>, <class 'float'>]"
1,app_name,"[<class 'str'>, <class 'float'>]"
2,title,"[<class 'str'>, <class 'float'>]"
3,url,[<class 'str'>]
4,tags,"[<class 'str'>, <class 'float'>]"
5,reviews_url,"[<class 'str'>, <class 'float'>]"
6,specs,"[<class 'str'>, <class 'float'>]"
7,price,[<class 'float'>]
8,early_access,[<class 'float'>]
9,item_id,[<class 'float'>]


Del DataFrame **'user_items'**, se extraen las columnas: *'items_count', 'user_id', 'item_id'*.

In [27]:
items = user_items[['items_count', 'user_id', 'item_id']]

Del DataFrame **'steam_games'**, se extraen las columnas: *'price', 'id'*.

In [28]:
games = steam_games[['price', 'item_id']]

In [29]:
games = games.drop_duplicates(subset = 'item_id', keep = 'first')

In [30]:
games

,price,item_id
0,4.99,761140.0
5,0.00,643980.0
9,0.00,670290.0
14,0.99,767400.0
17,2.99,773570.0
...,...,...
74821,1.99,773640.0
74825,4.99,733530.0
74828,1.99,610660.0
74831,4.99,658870.0


Cambiamos el tipo de dato de la columna **'item_id'** de *float* a *int*:

In [31]:
games['item_id'] = pd.to_numeric(games['item_id'], errors='coerce').fillna(0).astype(int)


Verificamos:

In [32]:
f.tipo_datos(games)

,nombre_columna,tipo_dato
0,price,[<class 'float'>]
1,item_id,[<class 'int'>]


Unimos ambos DataFrames con las columnas necesarias:

In [33]:
df_gasto = items.merge(games, on = 'item_id', how = 'left')

Analizamos los valores nulos de cada columna del nuevo DataFrame:

In [34]:
f.porcentaje_valores_nulos(df_gasto)

La columna items_count tiene un  0.00 % de valores nulos
La columna user_id tiene un  0.00 % de valores nulos
La columna item_id tiene un  0.00 % de valores nulos
La columna price tiene un  16.67 % de valores nulos


Se reemplazan los valores Nan por 0.0, considerandolos juegos gratuitos:

In [35]:
df_gasto['price'] = df_gasto['price'].fillna(0.0)

Se elimina la columna 'item_id', ya que solo la utilizamos para concatenar ambos DataFrames:

In [36]:
df_gasto = df_gasto.drop(columns = 'item_id')

Luego, agrupamos según usuario, la cantidad de dinero gastado y reindexamos:

In [37]:
df_gasto_agrupado = df_gasto.groupby('user_id')['price'].sum().reset_index()

Creamos otro DataFrame con los items por usuario:

In [38]:
df_items = df_gasto[['user_id', 'items_count']]

Eliminamos duplicados según id:

In [39]:
df_items.drop_duplicates(subset = 'user_id', keep = 'first', inplace = True)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3248\365604175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items.drop_duplicates(subset = 'user_id', keep = 'first', inplace = True)


Finalmente, unimos el DataFrame *'items'* (items por usuario), con el DataFrame *'gastos'* (total de dinero gastado por usuario):

In [40]:
df_userdata = df_items.merge(df_gasto_agrupado, on = 'user_id', how = 'right')

In [86]:
user_reviews = user_reviews[['user_id', 'recommend']]

In [87]:
def UserData(user_id):

    user = user_reviews[user_reviews['user_id'] == user_id]
    
    dinero_gastado = df_userdata[df_userdata['user_id'] == user_id]['price'].iloc[0]

    items_comprados = df_userdata[df_userdata['user_id'] == user_id]['items_count'].iloc[0]

    

    recomendaciones = user['recommend'].sum()

    reviews = len(user_reviews['user_id'])

    porcent_recomendaciones = ((recomendaciones / reviews) * 100)

    return {
        'user_id': user_id,
        'dinero_gastado': dinero_gastado,
        'items_comprados': items_comprados.astype(int),
        '%_recomendaciones': round(porcent_recomendaciones, 3)
    }




In [88]:
UserData('zzyfo')

{'user_id': 'zzyfo',
 'dinero_gastado': 828.51,
 'items_comprados': 84,
 '%_recomendaciones': 0.0}

In [90]:
user_reviews.to_parquet('Data/user_reviews.parquet', index=False)

In [89]:
df_userdata.to_parquet('Data/userdata.parquet', index = False)

#### Funcion **UserForGenre**

*def UserForGenre(genero: str):* Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

Analizamos las columnas de los DataFrames que necesitamos para nuestra función y el tipo de datos de cada columna:

In [43]:
f.tipo_datos(steam_games)

,nombre_columna,tipo_dato
0,publisher,"[<class 'str'>, <class 'float'>]"
1,app_name,"[<class 'str'>, <class 'float'>]"
2,title,"[<class 'str'>, <class 'float'>]"
3,url,[<class 'str'>]
4,tags,"[<class 'str'>, <class 'float'>]"
5,reviews_url,"[<class 'str'>, <class 'float'>]"
6,specs,"[<class 'str'>, <class 'float'>]"
7,price,[<class 'float'>]
8,early_access,[<class 'float'>]
9,item_id,[<class 'float'>]


In [44]:
f.tipo_datos(user_items)

,nombre_columna,tipo_dato
0,item_id,[<class 'int'>]
1,item_name,[<class 'str'>]
2,playtime_forever,[<class 'int'>]
3,playtime_2weeks,[<class 'int'>]
4,user_id,[<class 'str'>]
5,items_count,[<class 'int'>]
6,steam_id,[<class 'int'>]
7,user_url,[<class 'str'>]


Extraemos las columnas necesarias de cada DataFrame:

In [45]:
genres = steam_games[['genres', 'release_year', 'item_id']]

In [46]:
genres

,genres,release_year,item_id
0,Action,2018.0,761140.0
1,Casual,2018.0,761140.0
2,Indie,2018.0,761140.0
3,Simulation,2018.0,761140.0
4,Strategy,2018.0,761140.0
...,...,...,...
74829,Racing,2018.0,610660.0
74830,Simulation,2018.0,610660.0
74831,Casual,2017.0,658870.0
74832,Indie,2017.0,658870.0


In [47]:
playtime = user_items[['user_id','playtime_forever', 'item_id']]

Cambiamos el tipo de dato de la columna 'item_id' para poder unir ambos DataFrames:

In [48]:
genres['item_id'] = pd.to_numeric(genres['item_id'], errors='coerce').fillna(0).astype(int)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3248\908956530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres['item_id'] = pd.to_numeric(genres['item_id'], errors='coerce').fillna(0).astype(int)


In [49]:
df_userforgenre = genres.merge(playtime, on = 'item_id')

In [50]:
df_userforgenre.drop(columns = 'item_id', inplace = True)

In [51]:
df_userforgenre = df_userforgenre.dropna(subset= ['release_year'])

In [52]:
df_userforgenre['release_year'].unique()

array([1997., 1998., 2006., 2005., 2003., 2007., 2002., 2000., 1995.,
       1996., 1994., 2001., 1993., 2004., 1999., 2008., 2009., 1992.,
       1989., 2010., 2011., 2013., 2012., 2014., 2017., 1983., 1984.,
       2015., 2016., 1990., 1988., 1991., 1987., 2018., 2024.])

In [53]:
df_userforgenre['release_year'] = df_userforgenre['release_year'].astype(int)


In [54]:
f.tipo_datos(df_userforgenre)

,nombre_columna,tipo_dato
0,genres,"[<class 'str'>, <class 'float'>]"
1,release_year,[<class 'int'>]
2,user_id,[<class 'str'>]
3,playtime_forever,[<class 'int'>]


In [55]:
df_userforgenre = df_userforgenre.sample(frac=0.05,random_state=5).reset_index(drop=True)

In [56]:
df_userforgenre.shape

(494060, 4)

In [57]:
def UserForGenre(genre):

    
    genre_df = df_userforgenre[df_userforgenre['genres'] == genre]

    
    genre_df['playtime_forever'] = (genre_df['playtime_forever'] / 60 / 60).astype(int)

    max_playtime_user = genre_df.loc[genre_df['playtime_forever'].idxmax(), 'user_id']

    
    yearly_playtime = genre_df.groupby('release_year')['playtime_forever'].sum().reset_index()

    
    playtime_list = [{'Año': int(year), 'Horas': int(hours)} for year, hours in zip(yearly_playtime['release_year'], yearly_playtime['playtime_forever'])]


    return {
        "Usuario con más horas jugadas para género " + genre: max_playtime_user, "Horas jugadas": playtime_list
        }


In [58]:
UserForGenre('Indie')

C:\Users\ferna\AppData\Local\Temp\ipykernel_3248\3496154921.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_df['playtime_forever'] = (genre_df['playtime_forever'] / 60 / 60).astype(int)


{'Usuario con más horas jugadas para género Indie': '76561198063648921',
 'Horas jugadas': [{'Año': 1988, 'Horas': 0},
  {'Año': 1995, 'Horas': 0},
  {'Año': 1996, 'Horas': 0},
  {'Año': 1997, 'Horas': 0},
  {'Año': 1998, 'Horas': 0},
  {'Año': 1999, 'Horas': 3},
  {'Año': 2000, 'Horas': 0},
  {'Año': 2001, 'Horas': 0},
  {'Año': 2002, 'Horas': 0},
  {'Año': 2003, 'Horas': 5},
  {'Año': 2004, 'Horas': 0},
  {'Año': 2005, 'Horas': 1},
  {'Año': 2006, 'Horas': 5059},
  {'Año': 2007, 'Horas': 9},
  {'Año': 2008, 'Horas': 31},
  {'Año': 2009, 'Horas': 4},
  {'Año': 2010, 'Horas': 27},
  {'Año': 2011, 'Horas': 2112},
  {'Año': 2012, 'Horas': 637},
  {'Año': 2013, 'Horas': 1340},
  {'Año': 2014, 'Horas': 367},
  {'Año': 2015, 'Horas': 1249},
  {'Año': 2016, 'Horas': 472},
  {'Año': 2017, 'Horas': 1044},
  {'Año': 2018, 'Horas': 0}]}

In [83]:
df_userforgenre.to_parquet('Data/userforgenre.parquet', index=False)

#### Función **BestDeveloperYear**

*def BestDeveloperYear(año : int):* Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Extraemos las columnas necesarias de cada DataFrame:

In [59]:
reviews = user_reviews[['item_id', 'recommend', 'sentiment_analysis']]

In [60]:
developer = steam_games[['item_id', 'release_year', 'developer']]

In [61]:
developer['item_id'] = pd.to_numeric(developer['item_id'], errors='coerce').fillna(0).astype(int)

C:\Users\ferna\AppData\Local\Temp\ipykernel_3248\2443650704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  developer['item_id'] = pd.to_numeric(developer['item_id'], errors='coerce').fillna(0).astype(int)


Unimos ambos DataFrames obtenidos anteriormente:

In [62]:
df_bestdeveloperyear = reviews.merge(developer, on = 'item_id').dropna()

Analizamos el tipo de datos de las columnas del nuevo DataFrame:

In [63]:
f.tipo_datos(df_bestdeveloperyear)

,nombre_columna,tipo_dato
0,item_id,[<class 'int'>]
1,recommend,[<class 'bool'>]
2,sentiment_analysis,[<class 'int'>]
3,release_year,[<class 'float'>]
4,developer,[<class 'str'>]


Eliminamos nulos según la columna 'release_year':

In [64]:
df_bestdeveloperyear.dropna(subset= ['release_year'])

,item_id,recommend,sentiment_analysis,release_year,developer
0,1250,True,2,2009.0,Tripwire Interactive
1,1250,True,2,2009.0,Tripwire Interactive
2,1250,True,2,2009.0,Tripwire Interactive
3,1250,True,1,2009.0,Tripwire Interactive
4,1250,True,1,2009.0,Tripwire Interactive
...,...,...,...,...,...
129764,262850,True,0,2014.0,SkyGoblin
129765,431510,True,2,2016.0,Aeon Dream Studios
129766,431510,True,2,2016.0,Aeon Dream Studios
129767,431510,True,2,2016.0,Aeon Dream Studios


Cambiamos el tipo de dato de float a int:

In [65]:
df_bestdeveloperyear['release_year'] = df_bestdeveloperyear['release_year'].astype(int)

In [66]:
f.tipo_datos(df_bestdeveloperyear)

,nombre_columna,tipo_dato
0,item_id,[<class 'int'>]
1,recommend,[<class 'bool'>]
2,sentiment_analysis,[<class 'int'>]
3,release_year,[<class 'int'>]
4,developer,[<class 'str'>]


In [67]:
df_bestdeveloperyear.drop(columns= 'item_id', inplace = True)

In [68]:
def BestDeveloperYear(year):
    
    anio = df_bestdeveloperyear[df_bestdeveloperyear['release_year'] == year]

    
    filtro = anio[(anio['sentiment_analysis'] == 2) & (anio['recommend'] == True)]

    
    developer = filtro.groupby('developer')['sentiment_analysis'].sum().reset_index()

    
    df_ordenado = developer.sort_values(by='sentiment_analysis', ascending=False)

    
    top_developers = df_ordenado.head(3)

    
    result = [{"Puesto {}: {}".format(i+1, row['developer'])} for i, (_, row) in enumerate(top_developers.iterrows())]

    return result

In [69]:
BestDeveloperYear(2017)

[{'Puesto 1: Smartly Dressed Games'},
 {'Puesto 2: Freejam'},
 {'Puesto 3: Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}]

In [79]:
df_bestdeveloperyear.to_parquet('Data/bestdeveloperyear.parquet')

#### Función **DeveloperReviewsAnalysis**

*def developer_reviews_analysis( desarrolladora: str):* Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

Extraemos las columnas necesarias de cada DataFrame:

In [70]:
developer = steam_games[['developer', 'item_id']].dropna()

In [71]:
reviews = user_reviews[['item_id', 'sentiment_analysis']].dropna()

Cambiamos el tipo de dato de la columna 'item_id', de float a int:

In [72]:
developer['item_id'] = developer['item_id'].astype(int)

Unimos ambos DataFrames:

In [73]:
df_developer_reviews = developer.merge(reviews, on = 'item_id')

Filtramos la columna 'sentiment_analysis', descartando las reseñas que se encuntren categorizadas como neutras:

In [74]:
df_developer_reviews = df_developer_reviews[df_developer_reviews['sentiment_analysis'] != 1]


Verificamos:

In [75]:
df_developer_reviews['sentiment_analysis'].unique()

array([2, 0], dtype=int64)

In [76]:
def DeveloperReviewsAnalysis(desarrollador):

    developer = df_developer_reviews[df_developer_reviews['developer'] == desarrollador]

    positivo = 0
    negativo = 0

    for sentiment in developer['sentiment_analysis']:
        if sentiment == 2:
            positivo += 1
        else:
            negativo += 1

    return {
        desarrollador: [f"Negative = {negativo}", f"Positive = {positivo}"]
    }

In [77]:
DeveloperReviewsAnalysis('Valve')

{'Valve': ['Negative = 1442', 'Positive = 6810']}

In [91]:
df_developer_reviews.to_parquet('Data/developer_reviews.parquet', index = False)